<a href="https://colab.research.google.com/github/gowtham91m/Python/blob/master/cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
import os, shutil
import pandas as pd
from time import time
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import layers,models,optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

root_dir = '/home/tensor/content/'
project_dir = os.path.join(root_dir,'cifar')
train_path = os.path.join(project_dir,'train')

batch_size=64

## data source 
## https://www.kaggle.com/c/cifar-10/data

### Data split using sklearn train test split

In [81]:
train_df = pd.read_csv(os.path.join(project_dir,'trainLabels.csv'))

X_train,X_val,y_train,y_val= train_test_split(train_df.id,train_df.label,test_size=0.25
                                              ,random_state=5,stratify=train_df.label)

train_data = pd.DataFrame({'id':X_train,'label':y_train}).reset_index().drop(['index'],axis=1)
val_data = pd.DataFrame({'id':X_val,'label':y_val}).reset_index().drop(['index'],axis=1)

datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(
                                          dataframe=train_data,
                                          directory=train_path,
                                          x_col="id",
                                          y_col="label",
                                          has_ext=False,    
                                          batch_size=32,
                                          seed=42,
                                          shuffle=True,
                                          class_mode="categorical",
                                          target_size=(32,32))
valid_generator=datagen.flow_from_dataframe(
                                          dataframe=val_data,
                                          directory=train_path,
                                          x_col="id",
                                          y_col="label",
                                          has_ext=False,
                                          batch_size=32,
                                          seed=42,
                                          shuffle=True,
                                          class_mode="categorical",
                                          target_size=(32,32))

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation=tf.nn.softmax))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Found 37500 images belonging to 10 classes.
Found 12500 images belonging to 10 classes.
Epoch 1/5
1171/1171 [==============================] - 14s 12ms/step - loss: 1.8775 - acc: 0.3108 - val_loss: 3.0206 - val_acc: 0.0938
Epoch 2/5
1171/1171 [==============================] - 13s 12ms/step - loss: 1.5783 - acc: 0.4268 - val_loss: 3.4161 - val_acc: 0.0816
Epoch 3/5
1171/1171 [==============================] - 14s 12ms/step - loss: 1.4440 - acc: 0.4756 - val_loss: 3.7098 - val_acc: 0.0780
Epoch 4/5
1171/1171 [==============================] - 14s 12ms/step - loss: 1.3488 - acc: 0.5165 - val_loss: 3.9940 - val_acc: 0.0623
Epoch 5/5
1171/1171 [==============================] - 14s 12ms/step - loss: 1.2728 - acc: 0.5459 - val_loss: 4.2509 - val_acc: 0.0511


### data split using subsetting in flow from dataframe

In [79]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)
train_generator=datagen.flow_from_dataframe(
                                          dataframe=train_df,
                                          directory=train_path,
                                          x_col="id",
                                          y_col="label",
                                          has_ext=False,                                      
                                          subset="training",
                                          batch_size=32,
                                          seed=42,
                                          shuffle=True,
                                          class_mode="categorical",
                                          target_size=(32,32))
valid_generator=datagen.flow_from_dataframe(
                                          dataframe=train_df,
                                          directory=train_path,
                                          x_col="id",
                                          y_col="label",
                                          has_ext=False,
                                          subset="validation",
                                          batch_size=32,
                                          seed=42,
                                          shuffle=True,
                                          class_mode="categorical",
                                          target_size=(32,32))

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation=tf.nn.softmax))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Found 37500 images belonging to 10 classes.
Found 12500 images belonging to 10 classes.
Epoch 1/5
1171/1171 [==============================] - 14s 12ms/step - loss: 1.8620 - acc: 0.3140 - val_loss: 1.6039 - val_acc: 0.4213
Epoch 2/5
1171/1171 [==============================] - 13s 11ms/step - loss: 1.5708 - acc: 0.4257 - val_loss: 1.4356 - val_acc: 0.4827
Epoch 3/5
1171/1171 [==============================] - 14s 12ms/step - loss: 1.4251 - acc: 0.4853 - val_loss: 1.3136 - val_acc: 0.5383
Epoch 4/5
1171/1171 [==============================] - 13s 11ms/step - loss: 1.3309 - acc: 0.5228 - val_loss: 1.2447 - val_acc: 0.5626
Epoch 5/5
1171/1171 [==============================] - 13s 12ms/step - loss: 1.2548 - acc: 0.5549 - val_loss: 1.1719 - val_acc: 0.5872
